#### Mining Massive Data Sets, Spring 2021

# Homework 1

In [ ]:
# pyspark --conf “spark.ui.port=10101”

!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 68kB/s 
     |████████████████████████████████| 204kB 44.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=4ac090f5500c006ea265e74b1b12f9befca7cbd40701fd1115a19ecb3e58c837
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 31 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Let's import the libraries we will need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext

In [ ]:
sc = SparkContext.getOrCreate()
sc.stop()

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

### Problem 1 -- Spark

In [ ]:
lines = sc.textFile("soc-LiveJournal1Adj.txt")
lines.take(5)

['0\t1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94',
 '1\t0,5,20,135,2409,8715,8932,10623,12347,12846,13840,13845,14005,20075,21556,22939,23520,28193,29724,29791,29826,30691,31232,31435,32317,32489,34394,35589,35605,35606,35613,35633,35648,35678,38737,43447,44846,44887,49226,49985,623,629,4999,6156,13912,14248,15190,17636,19217,20074,27536,29481,29726,29767,30257,33060,34250,34280,34392,34406,34418,34420,34439,34450,34651,45054,49592',
 '2\t0,117,135,1220,2755,12453,24539,24714,41456,45046,49927,6893,13795,16659,32828,41878',
 '3\t0,12,41,55,1532,12636,13185,27552,38737',
 '4\t0,8,14,15,18,27,72,80,15326,19068,19079,24596,42697,46126,74,77,33269,38792,38822']

In [ ]:
split = lines.map(lambda line: line.split())
split.take(5)

[['0',
  '1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94'],
 ['1',
  '0,5,20,135,2409,8715,8932,10623,12347,12846,13840,13845,14005,20075,21556,22939,23520,28193,29724,29791,29826,30691,31232,31435,32317,32489,34394,35589,35605,35606,35613,35633,35648,35678,38737,43447,44846,44887,49226,49985,623,629,4999,6156,13912,14248,15190,17636,19217,20074,27536,29481,29726,29767,30257,33060,34250,34280,34392,34406,34418,34420,34439,34450,34651,45054,49592'],
 ['2',
  '0,117,135,1220,2755,12453,24539,24714,41456,45046,49927,6893,13795,16659,32828,41878'],
 ['3', '0,12,41,55,1532,12636,13185,27552,38737'],
 ['4',
  '0,8,14,15,18,27,72,80,15326,19068,19079,24596,42697,46126,74,77,33269,38792,38822']]

In [ ]:
def make_network(line):
  splitted = line.split()
  ID = int(splitted[0])
  if len(splitted) == 1:
    friends = []
  else:
    friend_list = splitted[1].split(",")
    friends = list(map(lambda x: int(x), friend_list))
  return ID, friends

In [ ]:
def make_list_of_mutuals(network):
    ID = network[0]
    friends = network[1]

    mutual_friends = []

  # ID is a friend with every one in a friends list -> 0 sign
  # Have one mutual friend - ID -> 1 sign
    for friend1 in friends:
        min_ID = np.min([ID, friend1])
        max_ID = np.max([ID, friend1])
        mutual_friends.append(((min_ID, max_ID), 0))

        for friend2 in friends:
            if friend1 != friend2:
              min_friend = np.min([friend1, friend2])
              max_friend = np.max([friend1, friend2])
              mutual_friends.append(((min_friend, max_friend), 1))  

    return mutual_friends

In [ ]:
def top_sort(friends_list):
  if len(friends_list) > 10:
    sort = sorted(friends_list, key = lambda x : (- x[1], x[0]))[:10] #only top 10
  else:
    sort = sorted(friends_list, key = lambda x : (- x[1], x[0]))
  top_10 = list(map(lambda x: x[0], sort))
  return top_10

In [ ]:
network = lines.map(make_network)

In [ ]:
mutual_friends = network.flatMap(make_list_of_mutuals)
mutual_friends.cache()
# mutual_friends.take(5)

PythonRDD[4] at RDD at PythonRDD.scala:53

In [ ]:
# Filter out the ones that are already friends:
filtered_mutual_friends = mutual_friends.groupByKey() \
                                        .mapValues(list) \
                                        .filter(lambda count: 0 not in count[1])
# filtered_mutual_friends.take(5)

In [ ]:
# Count the number od mutual friends:
count_mutual_friends = filtered_mutual_friends.map(lambda pair: (pair[0], np.sum(pair[1])))
# count_mutual_friends.take(5)

In [ ]:
# Rearange the data so that we can group by key, where keys are all people in the network

friend_recommendations = count_mutual_friends.flatMap(lambda pair: [(pair[0][0], (pair[0][1], pair[1])), (pair[0][1], (pair[0][0], pair[1]))])
# friend_recommendations.take(5)

In [ ]:
all_connections = friend_recommendations.groupByKey().mapValues(list)

In [ ]:
top_10 = all_connections.map(lambda f: (f[0], top_sort(list(f[1]))))

In [ ]:
# top_10.coalesce(1).saveAsTextFile("top_10_recommendations.txt")

In [ ]:
l = top_10.collect()

In [ ]:
l_sorted = sorted(l, key = lambda x : x[0])

In [ ]:
f = open("top10_recommendations.txt", "w")
for element in l_sorted:
  f.write(str(element[0]) + "\t" + ",".join([str(el) for el in element[1]]) + "\n")
f.close()

### Problem 2 -- Association Rules

In [ ]:
lines = sc.textFile("browsing.txt")
lines.take(5)

['FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 ',
 'GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 ',
 'ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 ',
 'ELE17451 ELE37798 FRO86643 GRO56989 ELE23393 SNA11465 ',
 'ELE17451 SNA69641 FRO86643 FRO78087 SNA11465 GRO39357 ELE28573 ELE11375 DAI54444 ']

In [ ]:
baskets = lines.map(lambda line: line.split())
baskets.take(1)

[['FRO11987', 'ELE17451', 'ELE89019', 'SNA90258', 'GRO99222']]

In [ ]:
def createC1(data):
    """
    Create a list of unique items in transaction data.
    Represent each item as a set of length 1.
    """
    C1 = []
    for transaction in data:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    
    # create a set for each item in C1
    return [set(x) for x in C1]

In [ ]:
def createCk(Lk, k):
    """
    Create a list of candidates of length k.
    
    Arguments:
       Lk: a list of frequent itemsets
       k: the size of the itemsets
    
    """
    cand_list = []
    len_Lk = len(Lk)
    
    # join sets if first k-2 items are equal
    for i in range(len_Lk):
        for j in range(i+1, len_Lk):
            L1 = list(Lk[i])[:k-2]
            L2 = list(Lk[j])[:k-2]
            L1.sort()
            L2.sort()
            if L1==L2:
                cand_list.append(Lk[i] | Lk[j])
    
    return cand_list

In [ ]:
def scanD(data, Ck, min_support):
    """
    Scan through transaction data and return a list of candidates that meet
    the support threshold, and support data about the current candidates.
    
    Arguments:
       data: data set,
       Ck: a list of candidate sets
       min_support: the minimum support
    """
    count = {}
    for transaction in data:
        tr = set(transaction)
        for candidate in Ck:
            if candidate.issubset(tr):
                can = frozenset(candidate)
                if can not in count:
                    count[can] = 1
                else:
                    count[can] += 1
                    num_items = float(len(D))
    
    cand_list = []
    support_data = {}
    
    # calculate support for every itemset
    for key in count:
        support = count[key]
        
        # If the support meets the minimum support requirements, 
        # add it to the list of itemsets.
        if support >= min_support:
            cand_list.insert(0, key)
        support_data[key] = support
    
    return cand_list, support_data
  

In [ ]:
def confidence(elem):
    lst=[]
    conf1 = elem[1]/(freq_pair_counts.filter(lambda x: x[0]==elem[0][0]).collect())[0][1]
    conf2 = elem[1]/(freq_items_counts.filter(lambda x: x[0]==elem[0][1]).collect())[0][1]
    lst.append(((elem[0],elem[1]),conf1))
    lst.append(((elem[1],elem[0]),conf2))
    return(lst)

In [ ]:
min_support = 100

In [ ]:
data = baskets.collect()

In [ ]:
C1 = createC1(data)

In [ ]:
D = list(map(set, data))

In [ ]:
L1, support_data1 = scanD(D, C1, min_support)

In [ ]:
C2 = createCk(L1, k=2)

In [ ]:
L2, support_data2 = scanD(D, C2, min_support)

In [ ]:
len(L1)

647

In [ ]:
def conf2(L1, L2, support_data1, support_data2):
  """
  returns the list with elements
    ((X, Y), conf(X -> Y)) 
  for all pairs in L2
  """
  scores = []
  for pair in L2:
    supp_XY = support_data2[pair] 
    X = list(pair)[0]
    Y = list(pair)[1]
    supp_X = support_data1[frozenset({X})]
    supp_Y = support_data1[frozenset({Y})]
    scores.append([[X, Y], supp_XY / supp_X])
    scores.append([[Y, X], supp_XY / supp_Y])
  return scores

In [ ]:
scores2 = conf2(L1, L2, support_data1, support_data2)

In [ ]:
sorted_scores2 = sorted(scores2, key = lambda x : x[1], reverse = True)

In [ ]:
sorted_scores2[:5]

[[['DAI93865', 'FRO40251'], 1.0],
 [['GRO85051', 'FRO40251'], 0.999176276771005],
 [['GRO38636', 'FRO40251'], 0.9906542056074766],
 [['ELE12951', 'FRO40251'], 0.9905660377358491],
 [['DAI88079', 'FRO40251'], 0.9867256637168141]]

In [ ]:
C3 = createCk(L2, k=3)

In [ ]:
L3, support_data3 = scanD(D, C3, min_support)
len(L3)

233

In [ ]:
def conf3(L1, L2, L3, support_data1, support_data2, support_data3):
  """
  returns the list with elements
    (((X, Y), Z), conf((X, Y) -> Z) 
  for all pairs in L2
  """
  scores = []
  for tripple in L3:
    supp_XYZ = support_data3[tripple] 
    X = list(tripple)[0]
    Y = list(tripple)[1]
    Z = list(tripple)[2]
    supp_XY = support_data2[frozenset({X, Y})]
    supp_XZ = support_data2[frozenset({X, Z})]
    supp_YZ = support_data2[frozenset({Y, Z})]
    scores.append([[sorted((X, Y)), Z], supp_XYZ / supp_XY])
    scores.append([[sorted((X, Z)), Y], supp_XYZ / supp_XZ])
    scores.append([[sorted((Y, Z)), X], supp_XYZ / supp_YZ])
  return scores

In [ ]:
scores3 = conf3(L1, L2, L3, support_data1, support_data2, support_data3)

In [ ]:
sorted_scores3 = sorted(scores3, key = lambda x : (- x[1], x[0][0][0], x[0][0][1]))

In [ ]:
sorted_scores3[:5]

[[[['DAI23334', 'ELE92920'], 'DAI62779'], 1.0],
 [[['DAI31081', 'GRO85051'], 'FRO40251'], 1.0],
 [[['DAI55911', 'GRO85051'], 'FRO40251'], 1.0],
 [[['DAI62779', 'DAI88079'], 'FRO40251'], 1.0],
 [[['DAI75645', 'GRO85051'], 'FRO40251'], 1.0]]

### Problem 4 --  LSH for Approximate Near Neighbor Search

Can be found in file lsh.py